In [1]:
import pandas as pd, chardet
import geopandas as gpd
import warnings
warnings.filterwarnings("ignore")

## Загрузка данных

### Автодорожные и железнодорожные связи между МО

In [2]:
connection = pd.read_parquet('data/connection.parquet').drop_duplicates()
print(connection.shape)
connection.head()

(4348966, 4)


,territory_id_x,territory_id_y,distance,type
0,2,1,100.9,highway
1,3,1,36.1,highway
2,3,2,85.0,highway
3,4,3,44.4,highway
4,4,1,59.9,highway


##### Проверяем данные на
- прямые дубликаты (совпадает полностью строка) и
- "замаскированные": (`territory_id_x`, `territory_id_y`, `distanc`, `type`) = (`territory_id_y`, `territory_id_x`, `distanc`, `type`)

(спойлер: они есть)

In [3]:
print(len(connection))
print(len(connection.drop_duplicates()))

4348966
4348966


In [4]:
connections_set = set(
    zip(
        connection['territory_id_x'],
        connection['territory_id_y'],
        connection['distance'],
        connection['type']
    )
)

total_error = 0
examples = []
for idx, (x, y, d, t) in enumerate(zip(
    connection['territory_id_x'],
    connection['territory_id_y'],
    connection['distance'],
    connection['type']
)):
    if (y, x, d, t) in connections_set:
        total_error += 1
        if len(examples) < 10:
            row = connection.iloc[idx]
            examples.append({
                'territory_id_x': x,
                'territory_id_y': y,
                'distance': d,
                'type': t,
            })

print("Total error:", total_error)
print("Примеры таких строк (до 10):")
for ex in examples:
    print(ex)

Total error: 174088
Примеры таких строк (до 10):
{'territory_id_x': 1208, 'territory_id_y': 1493, 'distance': 291.0, 'type': 'railway'}
{'territory_id_x': 1208, 'territory_id_y': 2187, 'distance': 334.0, 'type': 'railway'}
{'territory_id_x': 1208, 'territory_id_y': 1732, 'distance': 354.0, 'type': 'railway'}
{'territory_id_x': 1208, 'territory_id_y': 2101, 'distance': 374.0, 'type': 'railway'}
{'territory_id_x': 1208, 'territory_id_y': 2119, 'distance': 374.0, 'type': 'railway'}
{'territory_id_x': 1208, 'territory_id_y': 1855, 'distance': 383.0, 'type': 'railway'}
{'territory_id_x': 1208, 'territory_id_y': 1856, 'distance': 398.0, 'type': 'railway'}
{'territory_id_x': 1208, 'territory_id_y': 1854, 'distance': 447.0, 'type': 'railway'}
{'territory_id_x': 1208, 'territory_id_y': 1712, 'distance': 471.0, 'type': 'railway'}
{'territory_id_x': 1208, 'territory_id_y': 1381, 'distance': 495.0, 'type': 'railway'}


In [5]:
connection[(connection['territory_id_x']==1493)&(connection['territory_id_y']==1208)&(connection['distance']==291)&(connection['type']=='railway')]

,territory_id_x,territory_id_y,distance,type
4687733,1493,1208,291.0,railway


In [6]:
connection[(connection['territory_id_x']==1208)&(connection['territory_id_y']==1493)&(connection['distance']==291)&(connection['type']=='railway')]

,territory_id_x,territory_id_y,distance,type
3303803,1208,1493,291.0,railway


In [7]:
print(f'{total_error/2/len(connection)*100:.2f} % строк лишние')

2.00 % строк лишние


In [8]:
# Убираем лишние строки

connection['min_id'] = connection.apply(lambda row: min(row['territory_id_x'], row['territory_id_y']), axis=1)
connection['max_id'] = connection.apply(lambda row: max(row['territory_id_x'], row['territory_id_y']), axis=1)
connection = connection.drop_duplicates(subset=['min_id', 'max_id', 'distance', 'type'], keep='first')
connection = connection.drop(columns=['min_id', 'max_id'])

### Потребление
- Данные за 2023 и 2024 года

In [9]:
consumption = pd.read_parquet('data/consumption.parquet')
print(consumption.shape)
consumption.head()

(303126, 4)


,date,territory_id,category,value
0,2023-01,1,Продовольствие,7692
1,2023-01,1,Здоровье,1271
2,2023-01,1,Маркетплейсы,2505
3,2023-01,1,Общественное питание,1142
5,2023-01,1,Транспорт,1718


In [10]:
consumption['category'].value_counts()

category
Продовольствие          50521
Здоровье                50521
Маркетплейсы            50521
Общественное питание    50521
Транспорт               50521
Все категории           50521
Name: count, dtype: int64

### Индекс доступности рынков на уровне МО
- Данные за 2024 год

In [11]:
market = pd.read_parquet('data/market_access.parquet')
print(market.shape)
market.head()

(2571, 2)


,territory_id,market_access
0,1,309.6
1,2,322.8
2,3,315.2
3,4,315.5
4,5,320.2


### Росстат население
- Данные за 2023 и 2024 года

In [12]:
pop = pd.read_parquet('data/rosstat/2_bdmo_population.parquet')
print(pop.shape)
pop.head()

(700215, 6)


,territory_id,year,period,age,gender,value
0,1402,2023,год,0,Женщины,288.0
1,1402,2023,год,0,Мужчины,301.0
2,1402,2023,год,1,Женщины,277.0
3,1402,2023,год,1,Мужчины,316.0
4,1402,2023,год,10,Женщины,414.0


In [13]:
pop['year'].value_counts()

year
2023    354988
2024    345227
Name: count, dtype: int64

In [14]:
pop['age'].unique()

array(['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18',
       '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28',
       '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38',
       '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48',
       '49', '5', '50', '51', '52', '53', '54', '55', '56', '57', '58',
       '59', '6', '60', '61', '62', '63', '64', '65', '65+', '66', '67',
       '68', '69', '7', '70+', '8', '9', 'Всего', '70', '71', '72', '73',
       '74', '75', '76', '77', '78', '79', '80+'], dtype=object)

### Росстат миграция
- Данные за 2023 год

In [15]:
migration = pd.read_parquet('data/rosstat/3_bdmo_migration.parquet')
print(migration.shape)
migration.head()

(106224, 6)


,territory_id,year,period,age,gender,value
0,2335,2023,год,75-79,Женщины,5.0
1,2335,2023,год,75-79,Мужчины,NaN
2,2335,2023,год,55-59,Женщины,-4.0
3,2335,2023,год,55-59,Мужчины,-4.0
4,2335,2023,год,15-19,Женщины,-11.0


In [16]:
migration['year'].unique()

array([2023])

### Росстат зарплаты
- Данные за 2023 и 2024 года

In [17]:
salary = pd.read_parquet('data/rosstat/4_bdmo_salary.parquet')
print(salary.shape)
salary.head()

(369812, 6)


,territory_id,year,period,okved_name,okved_letter,value
0,3,2023,январь-декабрь,Все отрасли,0,46265.2
1,3,2023,январь-декабрь,Обрабатывающие производства,C,50330.1
2,3,2023,январь-декабрь,Услуги ЖКХ,D,43988.1
3,3,2023,январь-декабрь,Строительство,F,78478.4
4,3,2023,январь-декабрь,Торговля,G,39262.9


In [18]:
salary[['year', 'period']].value_counts()

year  period         
2023  январь-декабрь     47005
      январь-сентябрь    46944
      январь-июнь        46895
      январь-март        46499
2024  январь-сентябрь    45768
      январь-июнь        45713
      январь-декабрь     45693
      январь-март        45295
Name: count, dtype: int64

### Мобильность

In [19]:
mobility = pd.read_excel('data/mobility_index.xlsx')
print(mobility.shape)
mobility.head()

(594, 4)


,territory_id,year,municipal_district_name,mobility_index_km
0,206,2024,городской округ Петрозаводский,2.865809
1,206,2025,городской округ Петрозаводский,2.994959
2,207,2024,городской округ Костомукшский,0.533957
3,207,2025,городской округ Костомукшский,0.474514
4,208,2024,Беломорский муниципальный округ,0.842636


In [20]:
mobility['year'].value_counts()

year
2024    297
2025    297
Name: count, dtype: int64

### Справочник МО

In [21]:
mo_list = pd.read_excel('data/t_dict_municipal_districts.xlsx')
print(mo_list.shape)
mo_list.head()

(3101, 18)


,municipal_district_name_short,oktmo,municipal_district_name,municipal_district_type,municipal_district_status,shape,shape_linked_oktmo,municipal_district_center,source_rosstat,year_from,year_to,territory_id,change_id_from,change_id_to,region_code,region_name,municipal_district_center_lat,municipal_district_center_lon
0,Майкоп,79-701-000-000,городской округ город Майкоп,городской округ,административный_центр_субъекта,1,NaN,г Майкоп,data-20180110-structure-20150128.csv,2018,9999,1,NaN,NaN,1,Республика Адыгея,44.606208,40.104053
1,Адыгейск,79-703-000-000,городской округ город Адыгейск,городской округ,NaN,1,NaN,г Адыгейск,data-20180110-structure-20150128.csv,2018,9999,2,NaN,NaN,1,Республика Адыгея,44.883378,39.190962
2,Гиагинский,79-605-000-000,Гиагинский муниципальный район,муниципальный район,NaN,1,NaN,ст-ца Гиагинская,data-20180110-structure-20150128.csv,2018,9999,3,NaN,NaN,1,Республика Адыгея,44.875585,40.056498
3,Кошехабльский,79-615-000-000,Кошехабльский муниципальный район,муниципальный район,NaN,1,NaN,аул Кошехабль,data-20180110-structure-20150128.csv,2018,9999,4,NaN,NaN,1,Республика Адыгея,44.896500,40.496363
4,Красногвардейский,79-618-000-000,Красногвардейский муниципальный район,муниципальный район,NaN,1,NaN,с Красногвардейское,data-20180110-structure-20150128.csv,2018,9999,5,NaN,NaN,1,Республика Адыгея,45.144356,39.586148


In [22]:
mo_geo = gpd.read_file('data/t_dict_municipal_districts_poly.gpkg')
print(mo_geo.shape)
mo_geo.head()

(2660, 6)


,osm_ref,osm_vers,territory_id,year_from,year_to,geometry
0,1749726,2023-01-01T01:00:00Z,462,2018,9999,"MULTIPOLYGON (((45.90308 43.32412, 45.90501 43..."
1,1957640,2023-01-01T01:00:00Z,461,2018,9999,"MULTIPOLYGON (((45.67362 43.20324, 45.67349 43..."
2,1749727,2023-01-01T01:00:00Z,463,2018,9999,"MULTIPOLYGON (((45.18361 42.8424, 45.18339 42...."
3,1749728,2023-01-01T01:00:00Z,464,2018,9999,"MULTIPOLYGON (((45.95342 42.67815, 45.95197 42..."
4,1749729,2023-01-01T01:00:00Z,465,2018,9999,"MULTIPOLYGON (((45.47566 43.25142, 45.47502 43..."


### Данные по Арктике
- Справочник для мэтчинга данных ПОРА с данными СберИндекса
- Данные ПОРА 2025

In [47]:
with open('data/territory_id_pora.csv', 'rb') as f:
    enc = chardet.detect(f.read(10000))['encoding']
print('detected encoding:', enc)

ter_pora = pd.read_csv('data/territory_id_pora.csv', encoding=enc, sep = ';')
pora = pd.read_excel('data/Data_SberIndex_POAD.xlsx')
pora = pora[pora['arctic']==True]
print(ter_pora.shape)
ter_pora.head()

detected encoding: windows-1251
(128, 8)


,region,municipality_up_name,municipality_up_name_actual,municipality_down_name,settlement_name,settlement_name_sep,type,territory_id
0,Архангельская область,Вельский муниципальный район,Вельский муниципальный район,Городское поселение Кулойское,рп Кулой,рп Кулой (Архангельская область),рп,897
1,Архангельская область,Виноградовский муниципальный район,Виноградовский муниципальный округ,Березниковское сельское поселение,посёлок Березник,посёлок Березник (Архангельская область),поселок,900
2,Архангельская область,Коношский муниципальный район,Коношский муниципальный район,Городское поселение Коношское,рп Коноша,рп Коноша (Архангельская область),рп,902
3,Архангельская область,Котласский муниципальный район,Котласский муниципальный округ,Городское поселение Приводинское,рп Приводино,рп Приводино (Архангельская область),рп,903
4,Архангельская область,Котласский муниципальный район,Котласский муниципальный округ,Городское поселение Шипицынское,рп Шипицыно,рп Шипицыно (Архангельская область),рп,903


In [48]:
len(pora['municipality_up_name'].unique())

34

In [49]:
len(pora.drop_duplicates())

46

### Смотрим на полноту данных СберИндекса для исследования по Арктическим НП

In [50]:
ter_pora = ter_pora[['municipality_up_name_actual',
          'settlement_name_sep',
       'region', 'territory_id']]
need_id = pd.merge(ter_pora, pora[['region', 'settlement_name_sep']],
                   how = 'inner', on = ['region', 'settlement_name_sep'])
need_id_lst =  list(set(need_id['territory_id']))
len(need_id_lst)

34

In [51]:
def to_find_conn(row):
    if row['territory_id_x'] in need_id_lst or row['territory_id_y'] in need_id_lst:
        return True
    else:
        return False
def to_find_id(row):
    if row['territory_id'] in need_id_lst:
        return True
    else:
        return False

need_connection = connection[connection.apply(to_find_conn, axis = 1)]
need_consumption = consumption[consumption.apply(to_find_id, axis = 1)]
need_market = market[market.apply(to_find_id, axis = 1)]
need_pop = pop[pop.apply(to_find_id, axis = 1)]
need_migration = migration[migration.apply(to_find_id, axis = 1)]
need_salary = salary[salary.apply(to_find_id, axis = 1)]
need_mobility = mobility[mobility.apply(to_find_id, axis = 1)]

print(f'filtered connection: {len(need_connection)} из {len(connection)}')
print(f'filtered consumption: {len(need_consumption)} из {len(consumption)}')
print(f'filtered market: {len(need_market)} из {len(market)}')
print(f'filtered pop: {len(need_pop)} из {len(pop)}')
print(f'filtered migration: {len(need_migration)} из {len(migration)}')
print(f'filtered salary: {len(need_salary)} из {len(salary)}')
print(f'filtered mobility: {len(need_mobility)} из {len(mobility)}')

filtered connection: 88123 из 4261922
filtered consumption: 4320 из 303126
filtered market: 28 из 2571
filtered pop: 9163 из 700215
filtered migration: 1322 из 106224
filtered salary: 4694 из 369812
filtered mobility: 30 из 594


#### Смотрим отстутствующие данные

In [52]:
connection_id_found = list(set(need_id_lst)&set(list(need_connection['territory_id_x'].unique())+list(need_connection['territory_id_y'].unique())))
consumption_id_found = list(need_consumption['territory_id'].unique())
market_id_found = list(need_market['territory_id'].unique())
pop_id_found = list(need_pop['territory_id'].unique())
migration_id_found = list(need_migration['territory_id'].unique())
salary_id_found = list(need_salary['territory_id'].unique())
mobility_id_found = list(need_mobility['territory_id'].unique())

print(f'filtered connection id: {len(connection_id_found)} из {len(need_id_lst)}')
print(f'filtered consumption id: {len(consumption_id_found)} из {len(need_id_lst)}')
print(f'filtered market id: {len(market_id_found)} из {len(need_id_lst)}')
print(f'filtered pop id: {len(pop_id_found)} из {len(need_id_lst)}')
print(f'filtered migration id: {len(migration_id_found)} из {len(need_id_lst)}')
print(f'filtered salary id: {len(salary_id_found)} из {len(need_id_lst)}')
print(f'filtered mobility id: {len(mobility_id_found)} из {len(need_id_lst)}')

filtered connection id: 29 из 34
filtered consumption id: 31 из 34
filtered market id: 28 из 34
filtered pop id: 34 из 34
filtered migration id: 34 из 34
filtered salary id: 34 из 34
filtered mobility id: 15 из 34


In [53]:
datasets = {
    'connection': connection_id_found,
    'consumption': consumption_id_found,
    'market': market_id_found,
    'pop': pop_id_found,
    'migration': migration_id_found,
    'salary': salary_id_found,
    'mobility': mobility_id_found,
}
dictt = {}
for ind in need_id_lst:
    for name, ds in datasets.items():
        if name == 'connection':
            dictt[ind] = {}
        if ind in ds:
            dictt[ind][name]=1
        else:
            dictt[ind][name]=0

In [54]:
pd.DataFrame(dictt).sum().value_counts()

7    14
6    12
4     5
5     3
Name: count, dtype: int64

#### Вывод:
- у 14 МО есть все данные
- у 12 МО отсутсвует 1 из наборов
- у 5 МО - 3 набора
- у 4 МО - 2 набора

В исследовании не будет использованы мобильность населения из-за большого количества пропусков

#### Смотрим миграцию
- Обрабатываем столбец `age` - приводим все к числам

In [ ]:
need_migration['age'] = need_migration['age'].apply(lambda x: int((int(x.split('-')[0])+int(x.split('-')[1]))/2) if '-' in x else x)
need_migration['age'] = need_migration['age'].apply(lambda x: int(x) if x!= 'Всего' else x)

In [154]:
need_migration[need_migration['age']=='Всего'].groupby(by = 'gender')['value'].describe()

,count,mean,std,min,25%,50%,75%,max
gender,,,,,,,,
Женщины,34.0,-2.823529,153.157568,-447.0,-46.0,-16.0,94.75,409.0
Мужчины,34.0,-21.735294,231.289967,-800.0,-42.5,-23.0,41.50,731.0


In [155]:
need_migration_sorted = need_migration[need_migration['age']!='Всего'].sort_values(by = 'age').dropna()
# # describe
# print('Describe')
# for migr in need_migration_sorted['age'].unique():
#     print(migr)
#     print(need_migration_sorted[need_migration_sorted['age']==migr].groupby(by = 'gender')['value'].describe())
#     print()

# # sum
# print('Sum')
# for migr in need_migration_sorted['age'].unique():
#     print(migr)
#     print(need_migration_sorted[need_migration_sorted['age']==migr].groupby(by = 'gender')['value'].sum())
#     print()

#### Скачиваем для дальнейшей визуализации

In [34]:
migr_for_mo = pd.merge(ter_pora[['region','municipality_up_name_actual',
                        'territory_id']], need_migration_sorted, on = 'territory_id',
              how = 'right')
migr_for_mo.to_excel('data/made/migration_for_mo.xlsx', index = False)

In [35]:
migr_for_mo.head()

,region,municipality_up_name_actual,territory_id,year,period,age,gender,value
0,Архангельская область,Пинежский муниципальный округ,910,2023,год,2,Женщины,-2.0
1,Мурманская область,Печенгский муниципальный округ,1521,2023,год,2,Мужчины,-22.0
2,Мурманская область,Печенгский муниципальный округ,1521,2023,год,2,Мужчины,-22.0
3,Чукотский автономный округ,Городской округ Эгвекинот,2345,2023,год,2,Женщины,8.0
4,Чукотский автономный округ,Городской округ Эгвекинот,2345,2023,год,2,Мужчины,11.0


### Разбиремся с данными ПОРА

In [55]:
pora = pd.merge(ter_pora[['territory_id', 'settlement_name_sep', 'municipality_up_name_actual']], pora,
                how = 'inner',
on = 'settlement_name_sep')

In [56]:
pora.shape

(46, 96)

In [57]:
# Смотрим пропуски

pora = pora[pora['arctic']==True]
miss = pora.isna().mean().sort_values()

print('Столбцы БЕЗ пропусков:')
print(miss[miss == 0].index.tolist(), '\n')

print('Столбцы с < 10 % пропусков:')
print(miss[miss < 0.1].index.tolist(), '\n')

print('Столбцы с 10–50 % пропусков:')
print(miss[(miss >= 0.1) & (miss <= 0.5)].index.tolist(), '\n')

print('Столбцы с > 50 % пропусков:')
print(miss[miss > 0.5].index.tolist())

Столбцы БЕЗ пропусков:
['territory_id', 'settlement_name_sep', 'municipality_up_name_actual', 'region', 'municipality_up_name', 'municipality_down_name', 'settlement_name', 'type', 'arctic', 'remote', 'special', 'suburb', 'latitude', 'pop_total', 'mun_oktmo', 'longitude', 'dtp_injury_norm', 'dtp_deaths_norm', 'sett_oktmo', 'wage_average', 'kindergarden_salary', 'school_salary', 'primary_avaiability', 'building_construction_space', 'natural_growth', 'sport_facilities', 'sport_halls', 'sport_pools', 'comments_health_perc', 'aviation_sun', 'aviation_center', 'death_rate', 'sport_stadiums', 'sport_gym', 'cinemas', 'libraries', 'certificate_fail', 'catering', 'culture_centers', 'theatres', 'retail_federal', 'ecology_polygon', 'comments_eco_perc', 'emissions_second', 'emissions_first', 'emissions_all', 'ecology_projects', 'ecology_spending', 'pvz_norm', 'nomadic_education_primary', 'nomadic_education_main', 'nomadic_education_high', 'revenue_per_capita', 'profit_bt_per_capita', 'loss_results

In [58]:
# Оставляем только признаки с пропусками менее 10 %
pora10 = pora[miss[miss < 0.1].index.tolist()]

### Добавляем признаки СберИндекса

#### Связность
- Считаем суммы авто и жд для каждого МО арктических НП из датасета ПОРА
- Итог: столбцы `railway_dist` и `highway_dist`

In [165]:
need_connection_aggr = pd.DataFrame(pd.concat([pd.DataFrame(need_connection.groupby(by=['territory_id_y',
                                                   'type'])['distance'].sum()).reset_index().rename(columns={'territory_id_y':'territory_id'}),
          pd.DataFrame(need_connection.groupby(by=['territory_id_x',
                                                   'type'])['distance'].sum()).reset_index().rename(columns={'territory_id_x':'territory_id'})], axis = 0).groupby(by=['territory_id',                                                                                                         
                                                   'type'])['distance'].sum()).reset_index()
pora10_ind = pd.merge(pora10,
                      need_connection_aggr[need_connection_aggr['type']=='highway'][['territory_id',
                                                                        'distance']], how = 'left',
         on ='territory_id').rename(columns={'distance':'highway_dist'})
pora10_ind = pd.merge(pora10_ind,
                      need_connection_aggr[need_connection_aggr['type']=='railway'][['territory_id',
                                                                        'distance']], how = 'left',
         on ='territory_id').rename(columns={'distance':'railway_dist'})

#### Потребление

Агрегируем данные по потреблению для каждого МО арктических НП из датасета ПОРА.
Для каждого МО записываем среднее по данной категории (за 2023-204 гг.), в итоге получаем следующие столбцы:
- `Все категории`,
- `Здоровье`,
- `Маркетплейсы`,
- `Общественное питание`,
- `Продовольствие`,
- `Транспорт`

In [138]:
import plotly.express as px

# Визуализация по категориям покупок
fig = px.histogram(need_consumption[need_consumption['category']!='Все категории'],
                   x = 'date', y = 'value', color = 'category', histfunc='avg')
fig.show(renderer='browser')

In [166]:
pivot = (pd.DataFrame(need_consumption.groupby(['territory_id', 'category'])['value'].mean()).reset_index()
         .pivot_table(
             index='territory_id',
             columns='category',
             values='value',
             aggfunc='mean')
         .reset_index()
         .rename_axis(None, axis=1)
)

pora10_ind = pd.merge(pora10_ind, pivot, how = 'left', on = 'territory_id')

#### Индекс доступности рынков на уровне МО

In [167]:
pora10_ind = pd.merge(pora10_ind, need_market, how = 'left', on = 'territory_id')

#### Росстат население

In [168]:
print(pop.shape)
pop.head()

(700215, 6)


,territory_id,year,period,age,gender,value
0,1402,2023,год,0,Женщины,288.0
1,1402,2023,год,0,Мужчины,301.0
2,1402,2023,год,1,Женщины,277.0
3,1402,2023,год,1,Мужчины,316.0
4,1402,2023,год,10,Женщины,414.0


- В данных по населению есть небольшая путаница: суммарное население из данных росстата по всем territory_id, которые там есть, различается если смотреть сразу по всем возрастам, `age` = `Всего` и если суммировать по-отельности `age` != `Всего`.

In [169]:
print(f"{int(pop[pop['age']!='Всего']['value'].sum())} - количество человек по фильтру `Всего`")
print(f"{int(pop[pop['age']=='Всего']['value'].sum())} - количество человек, если считать по всем возрастам в age, исключая `Всего`")

304188436 - количество человек по фильтру `Всего`
268691650 - количество человек, если считать по всем возрастам в age, исключая `Всего`


- Также в `age` есть не числовые значения по типу `70+`, `65+` - уберем их полностью (если оставить, то количество человек, если считать по всем возрастам в `age`, исключая `Всего` будет превышать количество человек по фильтру `Всего`, что странно и вероятно ошибочно, поэтому уберем эти возрастные категории полностью
- В уникальных значениях видно, что возраста от `65` до `80` есть, но возрастов более `80` - нет, заменим все `age`=`80+` на `80`

In [170]:
pop_fix = pop[(pop['age']!='Всего')&(pop['age']!='65+')&(pop['age']!='70+')]
pop_fix['age_fix'] = pop_fix['age'].apply(lambda x: int(str(x).replace('+','')))
pop_fix['age_fix'].unique()

array([ 0,  1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,  2, 20, 21, 22, 23,
       24, 25, 26, 27, 28, 29,  3, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39,
        4, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49,  5, 50, 51, 52, 53, 54,
       55, 56, 57, 58, 59,  6, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69,  7,
        8,  9, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80])

In [171]:
pop_fix = pop[(pop['age']!='Всего')&(pop['age']!='65+')&(pop['age']!='70+')]
pop_fix['age_fix'] = pop_fix['age'].apply(lambda x: int(str(x).replace('+','')))
int(pop_fix['value'].sum())
print(f"{int(pop_fix['value'].sum())} - количество человек по всем возрастам еще уменьшилась")

240266277 - количество человек по всем возрастам еще уменьшилась


Считаем средневзвешенный возраст на каждый `territory_id` и `год`
- Для всех
- Для мужчин
- Для женщин

In [172]:
pop_fix['agg_sum'] = pop_fix['age_fix']*pop_fix['value']
pop_fix2 = pop_fix.groupby(['territory_id','year'])[['agg_sum', 'value']].sum().reset_index()
pop_fix2['avg_age'] = round(pop_fix2.apply(lambda row:row['agg_sum']/row['value'] if row['value']!=0 else 0,
                                           axis = 1), 2)
pop_fix2= pop_fix2.drop('value', axis= 1)

pop_fix_men = pop_fix[pop_fix['gender']=='Мужчины']
pop_fix2_men = pop_fix_men.groupby(['territory_id','year'])[['agg_sum', 'value']].sum().reset_index()
pop_fix2_men['avg_age_men'] = round(pop_fix2_men.apply(lambda row:row['agg_sum']/row['value'] if row['value']!=0 else 0,
                                           axis = 1), 2)
pop_fix2_men= pop_fix2_men.drop('value', axis= 1)

pop_fix_women = pop_fix[pop_fix['gender']=='Женщины']
pop_fix2_women = pop_fix_women.groupby(['territory_id','year'])[['agg_sum', 'value']].sum().reset_index()
pop_fix2_women['avg_age_women'] = round(pop_fix2_women.apply(lambda row:row['agg_sum']/row['value'] if row['value']!=0 else 0,
                                           axis = 1), 2)
pop_fix2_women= pop_fix2_women.drop('value', axis= 1)

Считаем населения на каждый `territory_id` и `год`
- Все
- Мужчины
- Женщины

Соединяем данные со средним возрастом и населением

In [173]:
pop_res = pd.merge(pop_fix2,
         pop_fix2_women[['territory_id',
                         'year',
                         'avg_age_women']],
         how = 'left', on =['territory_id', 'year'])
pop_res = pd.merge(pop_res,
         pop_fix2_men[['territory_id',
                       'year',
                       'avg_age_men']],
         how = 'left', on =['territory_id', 'year'])
pop_res = pd.merge(pop_res,
         pd.DataFrame(pop[pop['age']=='Всего'].groupby(['territory_id','year'])['value'].sum()).reset_index(),
         how = 'left', on =['territory_id', 'year'])
pop_res['pop_total_rosstat']=pop_res['value']
pop_res= pop_res.drop('value', axis= 1)

pop_res = pd.merge(pop_res,
         pd.DataFrame(pop[(pop['age']=='Всего')&(pop['gender']=='Мужчины')].groupby(['territory_id','year'])['value'].sum()).reset_index(),
         how = 'left', on =['territory_id', 'year'])
pop_res['pop_men_rosstat']=pop_res['value']
pop_res= pop_res.drop('value', axis= 1)

pop_res = pd.merge(pop_res,
         pd.DataFrame(pop[(pop['age']=='Всего')&(pop['gender']=='Женщины')].groupby(['territory_id','year'])['value'].sum()).reset_index(),
         how = 'left', on =['territory_id', 'year'])
pop_res['pop_women_rosstat']=pop_res['value']
pop_res= pop_res.drop('value', axis= 1)

Присоединяем к общему датасету получившиеся признаки
- `avg_age_2023`
- `avg_age_women_2023`
- `avg_age_men_2023`
- `pop_total_rosstat_2023`
- `pop_men_rosstat_2023`
- `pop_women_rosstat_2023`
- `avg_age_2024`
- `avg_age_women_2024`
- `avg_age_men_2024`
- `pop_total_rosstat_2024`
- `pop_men_rosstat_2024`
- `pop_women_rosstat_2024`

In [174]:
pop_res_2023 = pop_res[pop_res['year']==2023].drop(['year','agg_sum'], axis = 1)
pop_res_2023 = pop_res_2023.rename(columns={'pop_total_rosstat':'pop_total_rosstat_2023',
                                            'avg_age':'avg_age_2023',
                                            'pop_women_rosstat':'pop_women_rosstat_2023',
                                            'avg_age_women':'avg_age_women_2023',
                                            'pop_men_rosstat':'pop_men_rosstat_2023',
                                            'avg_age_men':'avg_age_men_2023'})

pop_res_2024 = pop_res[pop_res['year']==2024].drop(['year','agg_sum'], axis = 1)
pop_res_2024 = pop_res_2024.rename(columns={'pop_total_rosstat':'pop_total_rosstat_2024',
                                            'avg_age':'avg_age_2024',
                                            'pop_women_rosstat':'pop_women_rosstat_2024',
                                            'avg_age_women':'avg_age_women_2024',
                                            'pop_men_rosstat':'pop_men_rosstat_2024',
                                            'avg_age_men':'avg_age_men_2024'})

pora10_ind = pd.merge(pora10_ind, pop_res_2023, how = 'left', on = 'territory_id')
pora10_ind = pd.merge(pora10_ind, pop_res_2024, how = 'left', on = 'territory_id')

In [184]:
pora10_ind['pop_men_share'] = round(pora10_ind['pop_men']/pora10_ind['pop_total'], 2)

#### Росстат миграция
Считаем средневзвешенный возраст отдельно для мужчин и женщин, у которых миграция > 0 и у которых < 0, а также суммарную миграцию по полу (через `age` = `Всего`)

- `2023plus_avg_age_women` - средневзвешенный возраст женщин с положительной миграцией
- `2023minus_avg_age_women` - средневзвешенный вораст женщин с отрицательной миграцией
- `2023migration_women` -  количество уехавших женщин
- `2023plus_avg_age_men` - средневзвешенный возраст мужчин с положительной миграцией
- `2023minus_avg_age_men` - средневзвешенный возраст мужчин с отрицательной миграцией
- `2023migration_men` - количество уехавших мужчин

In [175]:
# Если выше не была проведена обработка `age`
# need_migration['age'] = need_migration['age'].apply(lambda x: int((int(x.split('-')[0])+int(x.split('-')[1]))/2) if '-' in x else x)
# need_migration['age'] = need_migration['age'].apply(lambda x: int(x) if x!= 'Всего' else x)
need_migration=need_migration.dropna()
migration_fix = need_migration[need_migration['age']!='Всего']
migration_fix['aggr_sum']=migration_fix['age']*need_migration['value']
migration_fix['aggr_sum_plus'] = migration_fix['aggr_sum'].apply(lambda x: x if x>0 else 0)
migration_fix['aggr_sum_minus'] = migration_fix['aggr_sum'].apply(lambda x: x if x<0 else 0)
migration_fix['value_plus'] = migration_fix['value'].apply(lambda x: x if x>0 else 0)
migration_fix['value_minus'] = migration_fix['value'].apply(lambda x: x if x<0 else 0)
migration_fix2 = pd.DataFrame(migration_fix.groupby(['territory_id', 'gender'])[['aggr_sum_plus',
                                                 'aggr_sum_minus',
                                                 'value_plus',
                                                 'value_minus']].sum()).reset_index()
migration_fix2['2023plus_avg_age'] = round(migration_fix2['aggr_sum_plus']/migration_fix2['value_plus'], 2)
migration_fix2['2023minus_avg_age'] = round(migration_fix2['aggr_sum_minus']/migration_fix2['value_minus'], 2)
migration_res = migration_fix2[['territory_id', 'gender', '2023plus_avg_age', '2023minus_avg_age']]
migration_res = pd.merge(migration_res,
         need_migration[need_migration['age']=='Всего'].rename(columns={'value':'migration'})[['territory_id',
                                                                                               'gender',
                                                                                               'migration']],
         on = ['territory_id', 'gender'], how = 'left')

migration_res_women = migration_res[migration_res['gender']=='Женщины']
migration_res_women = migration_res_women.rename(columns={'2023plus_avg_age':'2023plus_avg_age_women',
                                                          '2023minus_avg_age': '2023minus_avg_age_women',
                                                          'migration': '2023migration_women'})

migration_res_men = migration_res[migration_res['gender']=='Мужчины']
migration_res_men = migration_res_men.rename(columns={'2023plus_avg_age':'2023plus_avg_age_men',
                                                      '2023minus_avg_age': '2023minus_avg_age_men',
                                                      'migration': '2023migration_men'})
migration_res = pd.merge(migration_res_women, migration_res_men, on = 'territory_id', how = 'outer').drop(['gender_x','gender_y'], axis = 1)
pora10_ind = pd.merge(pora10_ind, migration_res, how = 'left', on = 'territory_id')

#### Росстат зарплата
Считаем среднюю зп для каждого МО арктических НП по отраслям занятости, а также минимальную и максимальную зарплату по МО

- `Все отрасли`
- `Административная деятельность`
- `Водоснабжение`
- `Гос. управление и военн. безопасность`
- `Гостиницы и общепит`
- `Добыча полезных ископаемых`
- `Здравоохранение`
- `ИТ и связь`
- `Научная и проф. деятельность`
- `Обрабатывающие производства`
- `Образование`
- `Спорт и досуг`
- `Строительство`
- `Торговля`
- `Транспортировка и хранение`
- `Услуги ЖКХ`
- `Финансы и страхование`
- `min_salary`
- `max_salary`

In [176]:
need_salary = need_salary.dropna()
need_salary.head()

,territory_id,year,period,okved_name,okved_letter,value
28188,221,2024,январь-декабрь,Все отрасли,0,79173.2
28189,221,2024,январь-декабрь,Услуги ЖКХ,D,79504.5
28190,221,2024,январь-декабрь,Строительство,F,107798.8
28191,221,2024,январь-декабрь,Торговля,G,58277.3
28192,221,2024,январь-декабрь,Транспортировка и хранение,H,66974.6


In [177]:
round(need_salary.groupby(by=['okved_name'])['value'].mean(), 2).sort_values(ascending=False)

okved_name
Добыча полезных ископаемых               180516.32
Транспортировка и хранение               122738.55
ИТ и связь                               121513.30
Научная и проф. деятельность             112073.99
Все отрасли                              109680.18
Строительство                            109340.48
Гос. управление и военн. безопасность    107551.76
Административная деятельность            107084.85
Финансы и страхование                    105423.96
Обрабатывающие производства              104665.87
Здравоохранение                           99747.83
Услуги ЖКХ                                95440.32
Спорт и досуг                             90337.22
Водоснабжение                             88844.69
Сельское хозяйство                        86615.64
Образование                               86066.36
Торговля                                  85178.31
Операции с недвижимостью                  83360.89
Гостиницы и общепит                       70896.39
Прочие услуги       

In [179]:
pivot = (pd.DataFrame(need_salary.groupby(['territory_id', 'okved_name'])['value'].mean()).reset_index()
         .pivot_table(
             index='territory_id',
             columns='okved_name',
             values='value',
             aggfunc='mean')
         .reset_index()
         .rename_axis(None, axis=1)
)

salary_stats = (
    need_salary
    .groupby('territory_id')['value']
    .agg(
         min_salary='min',
         max_salary='max')
    .round(2)
).reset_index()

pora10_ind = pd.merge(pora10_ind, pivot, how = 'left', on = 'territory_id')
pora10_ind = pd.merge(pora10_ind, salary_stats, how = 'left', on = 'territory_id')

### Рассчитываем остаток от средней зп по всем отраслям за вычетом трат на продовольствие и товаров из категории "здоровье"

In [182]:
pora10_ind['rub_for_life']=round(pora10_ind['Все отрасли']-(pora10_ind['Здоровье']+pora10_ind['Продовольствие']))

In [185]:
print(pora10_ind.shape)
pora10_ind.to_excel('data/made/pora10_ind.xlsx', index = False)

(46, 131)
